In [1]:
data_var = '2023-12-20'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9178,2023-12-20,Brasil Nbb,19:00,Minas,Brasília,1.04,9.85,162.5,1.87,1.79,-21.5,2.00,1.72,Kj4evQGd,0.961538,0.101523,0.534759,0.558659,0.063061,0.6,0.2,NaN,NaN,108.708,23.355310,0.214844,360.076,240.106060,0.666821,108.474,425.718,70.35,409.53,0.0,0.0,1.0,0.0,1.144097,0.030912,0.106446,-1.20,-0.240,-0.166667,0.000000,0.0,0.000000,-5.00,-1.000,-8.850000,0.000000,0.0,0.000000
9179,2023-12-20,Brasil Nbb,19:30,São José,Bauru,1.45,2.62,155.5,1.85,1.85,-5.5,1.95,1.76,OOUymIKd,0.689655,0.381679,0.540541,0.540541,0.071335,0.6,0.8,NaN,NaN,119.684,18.160395,0.151736,110.024,14.103853,0.128189,187.518,178.672,98.28,126.72,0.0,0.0,0.0,0.0,0.406543,0.000000,0.072426,2.96,0.592,0.760135,0.000000,0.0,0.000000,-3.56,-0.712,-2.275281,0.000000,0.0,0.000000
9180,2023-12-20,Brasil Nbb,20:00,Botafogo,Unifacisa,3.20,1.32,158.5,1.85,1.83,5.5,1.94,1.77,2FTunxz3,0.312500,0.757576,0.540541,0.546448,0.070076,0.6,0.6,NaN,NaN,351.016,123.317149,0.351315,150.148,85.908939,0.572162,253.406,170.706,188.10,129.22,0.0,0.0,0.0,0.0,0.588213,0.007686,0.064802,2.35,0.470,4.680851,0.000000,0.0,0.000000,-3.50,-0.700,-0.457143,0.000000,0.0,0.000000
9181,2023-12-20,Eua Nba,00:00,Golden State Warriors,Boston Celtics,2.98,1.47,233.5,1.86,2.07,5.5,2.01,1.93,ADnVcafq,0.335570,0.680272,0.537634,0.483092,0.015843,0.8,0.4,NaN,NaN,245.710,60.865397,0.247712,164.140,20.598427,0.125493,195.800,164.348,203.36,145.92,0.0,0.0,0.0,0.0,0.479879,0.075569,0.028715,0.96,0.192,10.312500,0.622547,0.5,-0.122547,-0.74,-0.148,-3.175676,0.629264,0.7,0.070736
9182,2023-12-20,Eua Nba,00:00,Portland Trail Blazers,Phoenix Suns,3.44,1.38,229.5,1.80,2.08,6.5,2.06,1.87,OGjZdJ9k,0.290698,0.724638,0.555556,0.480769,0.015335,0.8,0.6,NaN,NaN,304.752,104.515581,0.342953,187.958,41.607089,0.221364,305.866,210.342,282.72,196.35,0.0,0.0,0.0,0.0,0.604415,0.102057,0.068372,-2.75,-0.550,-4.436364,0.376324,0.2,-0.176324,1.93,0.386,0.984456,0.518279,0.6,0.081721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9331,2023-12-20,Romênia Divisão A,13:00,CSM Constanta,SCM Craiova,1.44,2.65,150.5,1.87,1.79,-6.5,2.03,1.70,Ms9twlgQ,0.694444,0.377358,0.534759,0.558659,0.071803,0.0,0.0,NaN,NaN,172.976,84.180636,0.486661,168.324,87.903707,0.522229,0.000,0.000,111.78,311.71,0.0,0.0,0.0,0.0,0.418386,0.030912,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9332,2023-12-20,Romênia Liga Nacional Feminina,11:00,Olimpia Brasov F,Cluj Napoca F,2.68,1.43,138.5,1.86,1.84,1.5,2.01,1.68,KUIbcmDD,0.373134,0.699301,0.537634,0.543478,0.072435,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,426.96,380.24,1.0,0.0,0.0,0.0,0.430114,0.007644,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9333,2023-12-20,Rússia Liga Vtb United,11:30,Parma Perm,CSKA Moscow,4.52,1.21,158.5,1.82,1.92,8.5,2.08,1.71,2BR88F8f,0.221239,0.826446,0.549451,0.520833,0.047685,0.4,0.2,NaN,NaN,153.818,29.720289,0.193217,115.168,32.561897,0.282734,193.368,114.796,135.15,103.20,0.0,0.0,0.0,0.0,0.816937,0.037813,0.138063,2.45,0.490,7.183673,0.000000,0.0,0.000000,1.83,0.366,0.573770,0.000000,0.0,0.000000
9334,2023-12-20,Rússia Liga Vtb United,12:30,Samara,MBA Moscow,1.26,3.90,157.5,1.81,1.95,-10.5,2.08,1.71,bwLH6gw7,0.793651,0.256410,0.552486,0.512821,0.050061,0.0,0.0,NaN

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,00:00,Eua Nba,Golden State Warriors,Boston Celtics,233.5,1.86,1.0000,Over
4,00:00,Eua Nba,Portland Trail Blazers,Phoenix Suns,229.5,1.80,0.9999,Over
6,21:00,Eua Nba,Indiana Pacers,Charlotte Hornets,247.5,1.97,0.5523,Over
14,15:00,Europa Euroliga,Monaco,Panathinaikos,160.5,1.85,1.0000,Over
77,23:00,Eua Ncaa,Oklahoma,North Carolina,154.5,1.91,0.8404,Over
